This file contains all the required experiments 

Project: Conformal prediction for digital soil mapping

Author: Nafiseh Kakhani, University of Tuebingen

Date: 16/10/2023

### import packages

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# from sklearn.linear_model import QuantileRegressor
# from sklearn.metrics import mean_pinball_loss, mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg
import random
import numpy as np
np.warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import os
import sys
import random

### load data
Please load your input features + output as .csv file in the following:

In [ ]:
df = pd.read_csv('path\\toyour\\csv file')

# Convert the column to numeric, omitting non-double values
df['OC'] = pd.to_numeric(df['OC'], errors='coerce')

# Drop rows with NaN values (non-double values)
df.dropna(subset=['OC'], inplace=True)

inputs and output of the model

In [ ]:
X = df.iloc[:,:-2]
y = df.iloc[:,-2]

In [ ]:
from cqr import helper
from nonconformist.nc import RegressorNc
from nonconformist.nc import QuantileRegErrFunc

seeds = [123, 450, 45, 609, 700]
# seeds = [5660, 968, 111]

# random_state_train_test = seed
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)

# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(seed)
    
# desired miscoverage error
alpha = 0.1

# desired quanitile levels
quantiles = [0.05, 0.95]

# used to determine the size of test set
test_ratio = 0.2
train_ratio = 0.95 #this value indicates the train ratio with regard to the val/cal ratio


In [ ]:

def split_train_test(X, y, test_ratio):

    # Divide the dataset into test and train based on the test_ratio parameter
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
    
    # Let us keep the indices for test samples for further mapping
    keep_inds = x_test.index.tolist()
    point_id = df.loc[keep_inds, 'point_id']
    
    # Reshape the data
    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    x_test = np.asarray(x_test)
    y_test = np.asarray(y_test)
    
    # Compute input dimensions
    n_train = x_train.shape[0]
    in_shape = x_train.shape[1]
    
    # # Display basic information
    # print("Dimensions: train set (n=%d, p=%d) ; test set (n=%d, p=%d)" % 
    #       (x_train.shape[0], x_train.shape[1], x_test.shape[0], x_test.shape[1]))
    
    return x_train, x_test, y_train, y_test, point_id, n_train



In [ ]:
x_train, x_test, y_train, y_test, point_id, n_train = split_train_test(X, y, test_ratio)

In [ ]:

def preprocess_training_and_scaling(x_train, x_test, y_train, y_test, n_train):

    # Divide the data into proper training set and calibration set
    idx = np.random.permutation(n_train)
    split_point = int(np.floor(n_train * train_ratio))
    
    # Split the indices into training and calibration sets
    idx_train, idx_cal = idx[:split_point], idx[split_point:]

    # Zero mean and unit variance scaling
    scalerX = StandardScaler()
    scalerX = scalerX.fit(x_train[idx_train])

    # Scale
    x_train = scalerX.transform(x_train)
    x_test = scalerX.transform(x_test)
    
    # Display basic information
    print("Dimensions: train set (n=%d, p=%d) ; test set (n=%d, p=%d); cal/val set: (n=%d)" % 
          (x_train.shape[0], x_train.shape[1], x_test.shape[0], x_test.shape[1], len(idx_cal)))

    # Using log transformation to see whether the results are improved
    y_train = np.log(np.squeeze(y_train))
    y_test = np.log(np.squeeze(y_test))

    return x_train, x_test, y_train, y_test, idx_train, idx_cal


In [ ]:
#Example usage:
x_train, x_test, y_train, y_test, idx_train, idx_cal = preprocess_training_and_scaling(x_train, x_test, y_train, y_test, n_train)

In [ ]:
#########################################################
# Quantile random forests parameters
# (See QuantileForestRegressorAdapter class in helper.py)
#########################################################

# the number of trees in the forest
n_estimators = 1000

# the minimum number of samples required to be at a leaf node
# (default skgarden's parameter)
min_samples_leaf = 1

# the number of features to consider when looking for the best split
# (default skgarden's parameter)
max_features = x_train.shape[1] 

# target quantile levels
quantiles_forest = [quantiles[0]*100, quantiles[1]*100]

# use cross-validation to tune the quantile levels?
cv_qforest = True

# when tuning the two QRF quantile levels one may
# ask for a prediction band with smaller average coverage
# to avoid too conservative estimation of the prediction band
# This would be equal to coverage_factor*(quantiles[1] - quantiles[0])
coverage_factor = 0.85

# ratio of held-out data, used in cross-validation
cv_test_ratio = 0.05

# seed for splitting the data in cross-validation.
# Also used as the seed in quantile random forests function
cv_random_state = 1

# determines the lowest and highest quantile level parameters.
# This is used when tuning the quanitle levels by cross-validation.
# The smallest value is equal to quantiles[0] - range_vals.
# Similarly, the largest value is equal to quantiles[1] + range_vals.
cv_range_vals = 30

# sweep over a grid of length num_vals when tuning QRF's quantile parameters                   
cv_num_vals = 5

# Number of bootstrap samples
n_bootstraps = 20

In [ ]:
def create_and_save_dataframe(y_lower, y_upper, y_test, point_id, output_file):

    df_oc = pd.DataFrame({
        'lower_oc': np.exp(y_lower),
        'upper_oc': np.exp(y_upper),
        'predicted_oc': (np.exp(y_upper) + np.exp(y_lower)) / 2,
        'standard_uncertainty': (np.exp(y_upper) - np.exp(y_lower)) / np.mean(np.exp(y_upper) + np.exp(y_lower)),
        'test_oc': np.exp(y_test),
        'Point_ID': point_id
    })
    
    # Save the DataFrame as a CSV file
    df_oc.to_csv(output_file, index=False)

In [ ]:
def train_gradient_boosting_models(x_train, y_train, x_test, point_id):

    # Reference: https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html

    # Define a parameter grid for hyperparameter tuning
    param_grid = {
        'learning_rate': [0.05, 0.1],
        'n_estimators': [200, 300],
        'max_depth': [1, 2],
        'min_samples_split': [2, 3]
    }

    # Initialize GradientBoostingRegressor with loss="quantile"
    gbr = GradientBoostingRegressor(loss="quantile")

    # Set up GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(x_train[idx_cal], y_train[idx_cal])

    # Get the best hyperparameters from the grid search
    best_params = grid_search.best_params_

    # Train models with the best hyperparameters
    all_models = {}
    for alpha_gb in [0.05, 0.95]:
        gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha_gb, **best_params)
        all_models[f"q {alpha_gb}"] = gbr.fit(x_train[idx_train], y_train[idx_train])

    # Make predictions with the trained models
    y_lower = all_models["q 0.05"].predict(x_test)
    y_upper = all_models["q 0.95"].predict(x_test)

    return y_lower, y_upper, point_id


# Other version of the above function without regularization
def train_quantile_linear(x_train, y_train, x_test, point_id):

   all_models = {}
    

   for alpha_gb in [0.01, 0.99]:
        qr = QuantReg(y_train, x_train)
        result = qr.fit(q = alpha_gb)
        all_models["q %1.2f" % alpha_gb] = result
    

   y_lower = all_models["q 0.01"].predict(x_test)
   y_upper = all_models["q 0.99"].predict(x_test)

   return y_lower, y_upper, point_id

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Define a custom PyTorch module for quantile regression
class QuantileRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QuantileRegression, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.fc(x)

# Define the Pinball Loss function
def pinball_loss(y_true, y_pred, quantile):
    e = y_true - y_pred
    loss = torch.mean(torch.max(quantile * e, (quantile - 1) * e))
    return loss

In [ ]:
params_qforest = dict()
params_qforest["random_state"] = 3333333
params_qforest["min_samples_leaf"] = min_samples_leaf
params_qforest["n_estimators"] = n_estimators
params_qforest["max_features"] = x_train.shape[1] 

params_qforest["CV"] = cv_qforest
params_qforest["coverage_factor"] = coverage_factor
params_qforest["test_ratio"]= cv_test_ratio
params_qforest["random_state"]= cv_random_state
params_qforest["range_vals"] = cv_range_vals
params_qforest["num_vals"] = cv_num_vals

In [ ]:
# Specify the path to the directory you want to change to
new_directory_path = "your/direcotry/path"

# Change the current working directory
os.chdir(new_directory_path)

In [ ]:
for i, seed in enumerate(seeds):
  
  # Set random seeds for reproducibility
  random.seed(seed)
  np.random.seed(seed)

  print(f"Iteration {i}")
  
  #Split data into train/test sets
  x_train, x_test, y_train, y_test, point_id, n_train = split_train_test(X, y, test_ratio)
  # Now we split the data in to different train/cal stes:
  x_train, x_test, y_train, y_test, idx_train, idx_cal = preprocess_training_and_scaling(x_train, x_test, y_train, y_test, n_train)

  #########################################################
  # Neural Network wih a pinball loss
  #########################################################

  # Convert NumPy arrays to PyTorch tensors
  xnn_train = torch.from_numpy(x_train[idx_train]).float()
  ynn_train = torch.from_numpy(y_train[idx_train]).float()
  xnn_val = torch.from_numpy(x_train[idx_cal]).float()
  ynn_val = torch.from_numpy(y_train[idx_cal]).float()
  xnn_test = torch.from_numpy(x_test).float()
  ynn_test = torch.from_numpy(y_test).float()

  # Create DataLoaders for the training and validation data
  train_dataset = TensorDataset(xnn_train, ynn_train)
  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
  val_dataset = TensorDataset(xnn_val, ynn_val)
  val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

  # Train models for each quantile with early stopping on validation loss
  models = {}
  for q in quantiles:
    model = QuantileRegression(xnn_train.shape[1], 1)
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    best_val_loss = float('inf')
    patience = 5  # Number of epochs without improvement to wait before early stopping
    early_stopping_count = 0

    model.train()
    for epoch in range(20):
        for data in train_loader:
            inputs, targets = data
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = pinball_loss(targets, predictions, q)
            loss.backward()
            optimizer.step()
        
        # Calculate validation loss
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for val_data in val_loader:
                val_inputs, val_targets = val_data
                val_predictions = model(val_inputs)
                val_loss += pinball_loss(val_targets, val_predictions, q).item()
        
        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_count = 0
        else:
            early_stopping_count += 1
        
        if early_stopping_count >= patience:
            print(f"Early stopping at epoch {epoch}.")
            break
        
        model.train()
    
    models[q] = model

  # Make predictions for 0.05 and 0.95 quantiles for the test data
  quantile_predictions_test = {q: model(xnn_test).detach().numpy() for q, model in models.items()}

  # Extract the lower (0.05) and upper (0.95) quantile predictions
  y_lower = quantile_predictions_test[0.05][:, 0]
  y_upper = quantile_predictions_test[0.95][:, 0]

  output_file = f'test_NN_{i}.csv'
  create_and_save_dataframe(y_lower, y_upper, y_test, point_id, output_file)
  
  #########################################################
  # Random forests + Bootstrapping
  #########################################################

  # Initialize lists to store predictions for each bootstrap sample
  bootstrap_predictions = []

  # Standard bootstrap: Omit around 36.8% of samples in each iteration
  sample_size = int(0.632 * x_train.shape[0])  # 63.2% of training samples
    
  for _ in range(n_bootstraps):
        # Create a new bootstrap sample by randomly selecting samples with replacement
        bootstrap_indices = np.random.choice(x_train.shape[0], size=sample_size, replace=False)
        X_bootstrap = x_train[bootstrap_indices]
        y_bootstrap = y_train[bootstrap_indices]
        
        # Create and train a new model on the bootstrap sample
        bootstrap_model = RandomForestRegressor(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, random_state = 42)
        bootstrap_model.fit(X_bootstrap, y_bootstrap)
        
        # Make predictions on the test data using the bootstrap model
        y_bootstrap_pred = bootstrap_model.predict(x_test)
        
        # Append the predictions to the list
        bootstrap_predictions.append(y_bootstrap_pred)
    

  y_upper = np.max(bootstrap_predictions, axis=0)
  y_lower = np.min(bootstrap_predictions, axis=0)

  output_file = f'test_Boot_RF_{i}.csv'
  create_and_save_dataframe(y_lower, y_upper, y_test, point_id, output_file)

  #########################################################
  # Quantile random forests + Conformal predicition 
  #########################################################

  model = helper.QuantileForestRegressorAdapter(model = None,
                                              fit_params=None,
                                              quantiles=quantiles_forest,
                                              params = params_qforest)
                                              

  nc = RegressorNc(model, QuantileRegErrFunc())
  y_lower, y_upper = helper.run_icp(nc, x_train, y_train, x_test, idx_train, idx_cal, alpha)
  
  output_file = f'test_cqr_quantile_forest_{i}.csv'
  create_and_save_dataframe(y_lower, y_upper, y_test, point_id, output_file)
  
  #########################################################
  # Prediction Intervals for Gradient Boosting Regression 
  #########################################################

  y_lower, y_upper, point_id = train_gradient_boosting_models(x_train, y_train, x_test, point_id)
  output_file = f'test_quantile_GBoost_{i}.csv'
  create_and_save_dataframe(y_lower, y_upper, y_test, point_id, output_file)

  ########################################################
  # Prediction Intervals for Quntile linear
  ########################################################

  y_lower, y_upper, point_id = train_quantile_linear(x_train, y_train, x_test, point_id)
  output_file = f'test_quantile_linear_{i}.csv'
  create_and_save_dataframe(y_lower, y_upper, y_test, point_id, output_file)